1. Скачайте датасет MNIST.
2. Изучите выборку (какие там картинки, сколько картинок в каждом классе).
3. Реализуйте загрузку датасета в память с диска с помощью OpenCV. Использовать готовые реализации датасетов и даталоудеров торча запрещено. Пока можно не заморачиваться многопоточностью.
4. Реализуйте аугментацию данных: поворот картинки на определенное кол-во градусов (задается аргументом), зашумление картинки. Реализовывать на numpy. Использовать готовые решения тоже запрещено.
5. Реализуйте способ скормить это нейросети (нормализация, приведение к одному разрешению).
6. Напишите код нейросети. Использовать Sequential запрещено. Модель пишете руками, наследуясь от torch.nn.Module.
7. Реализуйте цикл обучения нейросети. Использовать готовые трейнеры и model.fit() запрещено. Кол-во эпох, learning rate и прочие параметры задаются аргументами.
8. Напишите код для инференса и оценки качества работы нейронной сети.
9. Оценка качества должна заключаться в подсчете Accuracy, Recall, Precision, F-мера. Пользоваться готовыми решениями тоже запрещено.
10. Оформите это как удобную библиотечку, которую можно склонить с гитхаба, положить куда надо файлы весов модели, выборку и обучить / инференсить модель.

In [1]:
import os
import urllib.request
import gzip
import cv2
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optimizer
import matplotlib.pyplot as plt

In [2]:
path = "./MNIST/archive/"
os.makedirs(path, exist_ok = True)
sizes = [784, 256, 10]
learning_rate = 0.005
epochs = 10


source = {"training_images" : "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
          "training_labels" : "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
          "test_images" : "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
          "test_labels" : "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
          }

In [3]:
data={}

for key in source:
  data[key] = source[key].split('/')[-1]
  if os.path.exists(path + data[key]):
    print(data[key] + " was here long before you were born...")
  else:
    urllib.request.urlretrieve(source[key], path + data[key])
    print(data[key] + " downloaded right now!")

train-images-idx3-ubyte.gz downloaded right now!
train-labels-idx1-ubyte.gz downloaded right now!
t10k-images-idx3-ubyte.gz downloaded right now!
t10k-labels-idx1-ubyte.gz downloaded right now!


In [4]:
dataset = {}
# Images
for key in ("training_images", "test_images"):
  with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
    dataset[key] = np.frombuffer(mnist.read(), np.uint8, offset = 16).reshape(-1, 28 * 28) / 255
    print("The shape of {}: {}".format(key, dataset[key].shape))

# Labels
for key in ("training_labels", "test_labels"):
  with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
    dataset[key] = np.frombuffer(mnist.read(), np.uint8, offset = 8)
    print("The shape of {}: {}".format(key, dataset[key].shape))


#print(dataset[key][0])

#Permutation
permutation = np.random.permutation(len(dataset[key]))
dataset[key] = dataset[key][permutation]

#print(permutation)

#print(dataset[key][0])

x_train, y_train, x_test, y_test = (
    dataset["training_images"],
    dataset["training_labels"],
    dataset["test_images"],
    dataset["test_labels"])

#print("The shape of {}: {}".format(key, dataset["training_images"].shape))
#print("The shape of {}: {}".format(key, dataset["training_labels"].shape))
#print("The shape of {}: {}".format(key, dataset["test_images"].shape))
#print("The shape of {}: {}".format(key, dataset["test_labels"].shape))

The shape of training_images: (60000, 784)
The shape of test_images: (10000, 784)
The shape of training_labels: (60000,)
The shape of test_labels: (10000,)


In [5]:
def noise(image):
  noise = np.random.normal(0.48, 0.05, image.shape)
  gaussedImage = np.clip(image + noise, 0, 1)
  return gaussedImage

def rotate(image):
  angle = np.random.randint(-30, 30)
  theta = np.radians(angle)
  cos, sin = np.cos(theta), np.sin(theta)
  height, width = image.shape
  MH, MW = height // 2, width // 2
  newImage = np.zeros(height, width)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      x = round((i - MW) * cos + (j - MH) * sin)
      y = round( - (i - MW) * sin + (j - MH) * cos)
      if (x >= 0 and y >= 0 and x < image.shape[0] and y < image.shape[1]):
        newImage[x, y] = image[i, j]
  return torch.tensor(newImage)

In [8]:
seed = 123456
gen = np.random.default_rng(seed)
learning_rate = 0.005
epochs = 10

def ReLu(self, x):
  return (x >= 0) * x

def dReLu(self, x):
  return x >= 0

def Softmax(self, x):
  return np.exp(x)/sum(np.exp(x))

def dSoftmax(x):
  return np.exp(x)/sum(np.exp(x)) * (1 - np.exp(x) / sum (np.exp(x)))


parameters = {
  'W1': 0.15 * gen.random((784, 256)) - 0.01 * np.sqrt(1 / 256),
  'W2': 0.15 * gen.random((256, 10)) - 0.01 * np.sqrt(1 / 10),
  'B1': 0.015 * gen.random(256) - 0.001,
  'B2': 0.015 * gen.random(10) - 0.001
  }

def forward(self, x_train):
  parameters = self.parameters
  parameters['A0'] = x_train
  parameters['L1'] = np.dot(parameters['A0'], parameters['W1']) + parameters['B1']
  parameters['A1'] = self.ReLu(parameters['L1'])
  parameters['D2'] = np.dot(parameters['A1'], parameters['W2']) + parameters['B2']
  parameters['A2'] = self.Softmax(parameters['D2'])
  return parameters['A2']

def backward(self, y_train, output):
  parameters = self.parameters

  updWeights = {}

  error = 2 * (output - y_train) / output.shape[0] * self.dSoftmax(parameters['L2'])
  updWeights['W2'] = np.outer(error, parameters['A1'])
  updWeights['B2'] = sum(((1 / self.batch) * error), axis = 0)
  error = np.dot(parameters['W2'].T, error) * self.dReLu(parameters['L1'])
  updWeights['B1'] = sum(np.dot(((1 / self.batch) * error), ['W2'].T) * self.dReLu(parameters['L1'], axis = 0))
  updWeights['W1'] = np.outer(error, parameters['A0'])
  accuracy += int(np.argmax(output, 1) == np.argmax(y_train, 1))
  for key, value in updWeights.items():
    self.updWeights[key] -= self.learning_rate * value
  return updWeights, accuracy, error


In [9]:
def train(self, x_train, y_train):
  for i in range(epochs):
    training_loss = []
    training_accuracy = []
    loss = 0.0
    accuracy = 0
    for j in range(len(x_train)):
      output = self.forward(x_train)
      updWeights, accuracy, loss = self.backward(y_train, output)

    training_loss.append(loss)
    training_accuracy.append(accuracy)

    print(( f"Epoch: {j}\n"
            f"  Training set loss: {loss / len(x_train):.3f}\n"
            f"  Training set accuracy: {accuracy / len(x_train)}"
            ))
  
#def test():
#true_positive = np.zeros()
#false_positive = np.zeros()
#false_negative = np.zeros()

#accuracy = correct/ len(labels_dataset)
#precision = true_positive / (true_positive + false_positive)
#Recall = true_positive / (true_positive + false_negative)
#F-Score = 2 * ((precision * recall) / (precision + recall))